Different types of tensors and their dimensions

In [ ]:
import numpy as np
import pandas as pd

a = 3 # a rank 0 tensor; this is a scalar with shape []
b = [1., 2., 3.] # a rank 1 tensor; this is a vector with shape [3]
c = [[1., 2., 3.], [4., 5., 6.]] # a rank 2 tensor; a matrix with shape [2, 3]
d = [[[1., 2., 3.]], [[7., 8., 9.]]] # a rank 3 tensor with shape [2, 1, 3]

a = np.array(a)
b = np.array(b)
c = np.array(c)
d = np.array(d)

print(a.shape, b.shape, c.shape, d.shape)

# Hello world! in TensorFlow

First few codes in the TensorFlow

In [ ]:
import tensorflow as tf

hello = tf.constant('Hello')
sess = tf.Session()
print(sess.run(hello))

In [ ]:
import tensorflow as tf

# Initialize two constants
vec1 = tf.constant([1,2,3,4])
vec2 = tf.constant([5,6,7,8])

addit = tf.add(vec1, vec2) # Addition
mult = tf.multiply(vec1, vec2) # Multiply

sess = tf.Session() # define session
print(sess.run(addit), sess.run(mult)) # Print the result
sess.close() # close tensorflow session

# TensorBoard

Adding names (labels) to tensors (a,b,c...) and group scopes to visualise tensor-graph sturcutre in TensorBoard (in conda prompt: tensorboard --logdir c:\\... and then in firefox or chrome: localhost:6006)

In [6]:
import tensorflow as tf

# reset everything to rerun in jupyter otherwise multiple old (earlier) graphs in graph section
tf.reset_default_graph()

with tf.name_scope('PreOperation'):
    with tf.name_scope('FirstTwo'):
        a = tf.add(1, 2, name='First_addition')
        b = tf.multiply(a, 3, name='First_multiplication')
    with tf.name_scope('SecondTwo'):
        c = tf.add(4, 5, name='Second_addition')
        d = tf.multiply(c, 6, name='Second_multiplication')

with tf.name_scope('MidOperation'):
    e = tf.multiply(4, 5, name='Another_multiplication')
    f = tf.div(c, 6, name='First_division')

with tf.name_scope('FinOperation'):
    g = tf.add(b, d, name='Last_addition')
    h = tf.multiply(g, f, name='Final_multiplication')

with tf.Session() as sess:
    writer = tf.summary.FileWriter('output3', sess.graph)
    #print(sess.run(h))
    writer.close()    

In [ ]:
import tensorflow as tf

h = tf.multiply(2, 3)
sess = tf.Session()

writer = tf.summary.FileWriter("C:\\Users\\PNaF\\0BigData\\output\\1", sess.graph)
a = sess.run(h)
aSummary = tf.summary.scalar('a',a)
writer.close()

In [ ]:
import tensorflow as tf

# reset everything to rerun in jupyter
tf.reset_default_graph()

# config
batch_size = 100
learning_rate = 0.5
training_epochs = 5
logs_path = "output"

# load mnist data set
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

# input images
with tf.name_scope('input'):
    # None -> batch size can be any size, 784 -> flattened mnist image
    x = tf.placeholder(tf.float32, shape=[None, 784], name="x-input") 
    # target 10 output classes
    y_ = tf.placeholder(tf.float32, shape=[None, 10], name="y-input")

# model parameters will change during training so we use tf.Variable
with tf.name_scope("weights"):
    W = tf.Variable(tf.zeros([784, 10]))

# bias
with tf.name_scope("biases"):
    b = tf.Variable(tf.zeros([10]))

# implement model
with tf.name_scope("softmax"):
    # y is our prediction
    y = tf.nn.softmax(tf.matmul(x,W) + b)

# specify cost function
with tf.name_scope('cross_entropy'):
    # this is our cost
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

# specify optimizer
with tf.name_scope('train'):
    # optimizer is an "operation" which we can execute in a session
    train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

with tf.name_scope('Accuracy'):
    # Accuracy
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
# create a summary for our cost and accuracy
tf.summary.scalar("cost", cross_entropy)
tf.summary.scalar("accuracy", accuracy)

# merge all summaries into a single "operation" which we can execute in a session 
summary_op = tf.summary.merge_all()

with tf.Session() as sess:
    # variables need to be initialized before we can use them
    sess.run(tf.global_variables_initializer())

    # create log writer object
    writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
        
    # perform training cycles
    for epoch in range(training_epochs):
        
        # number of batches in one epoch
        batch_count = int(mnist.train.num_examples/batch_size)
        
        for i in range(batch_count):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            
            # perform the operations we defined earlier on batch
            _, summary = sess.run([train_op, summary_op], feed_dict={x: batch_x, y_: batch_y})
            
            # write log
            writer.add_summary(summary, epoch * batch_count + i)
            
        if epoch % 5 == 0: 
            print("Epoch: ", epoch)
    print("Accuracy: ", accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))
    print("done")